In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the Excel file
excel_file_path = 'E:\\Correct_Data_Official\\Official_Images\\Official_volcano_sheet.xlsx'  # Path to the .xlsx file
data_txt = pd.read_excel(excel_file_path)

# Ensure the columns exist in the DataFrame
if not all(col in data_txt.columns for col in ['Gene_name', 'log2fc', 'padjusted']):
    raise ValueError("Expected columns ('Gene_name', 'log2fc', 'padjusted') were not found in the file.")

# Extract the relevant columns
log2FC = data_txt['log2fc']
transformed_p_value = data_txt['padjusted']  # Already in the -log10 scale

# Create figure with subplots for the Volcano Plot and KDE plots
fig = plt.figure(figsize=(12, 12))
grid = plt.GridSpec(4, 4, hspace=0.4, wspace=0.4)

# Main subplot (Volcano plot)
main_ax = fig.add_subplot(grid[1:4, 0:3])
main_ax.scatter(log2FC, transformed_p_value, color='grey', alpha=0.3, s=60, label='Not significant')

# Highlight points that meet the fold change and p-value criteria
highlighted_upregulated = (log2FC >= 0.263) & (transformed_p_value > -np.log10(0.05))  # Up-regulated
highlighted_downregulated = (log2FC <= -0.263) & (transformed_p_value > -np.log10(0.05))  # Down-regulated

main_ax.scatter(log2FC[highlighted_upregulated],
                transformed_p_value[highlighted_upregulated],
                color='#a30523', alpha=0.6, s=60, label='Up-regulated')

main_ax.scatter(log2FC[highlighted_downregulated],
                transformed_p_value[highlighted_downregulated],
                color='#6da0f2', alpha=0.6, s=60, label='Down-regulated')

# Add reference lines
main_ax.axhline(y=-np.log10(0.05), color='blue', linestyle='--', label='p-value threshold (0.05)')
main_ax.axvline(x=0.263, color='green', linestyle='--', label='Fold Change threshold (1.2)')
main_ax.axvline(x=-0.263, color='green', linestyle='--')

# Add legends and axis labels
main_ax.set_xlabel('Log2 (FC)')
main_ax.set_ylabel('-Log10 (p-Adjusted)')
main_ax.legend()
main_ax.set_title('')

# KDE above the main plot (for Log2(FC))
x_hist = fig.add_subplot(grid[0, 0:3], sharex=main_ax)
sns.kdeplot(log2FC, ax=x_hist, color='grey', fill=True, alpha=0.6)
x_hist.set_ylabel('Density')
x_hist.set_xlabel('')  # Updated x-axis label
x_hist.set_title('')

# KDE to the right of the main plot (for -Log10(p-value))
y_hist = fig.add_subplot(grid[1:4, 3], sharey=main_ax)
sns.kdeplot(transformed_p_value, ax=y_hist, color='grey', fill=True, alpha=0.6, vertical=True)
y_hist.set_xlabel('Density')
y_hist.set_ylabel('')  # Updated y-axis label
y_hist.set_title('')

# Remove unnecessary ticks
x_hist.tick_params(axis='x', which='both', bottom=False, labelbottom=True)
y_hist.tick_params(axis='y', which='both', left=False, labelleft=False)

# Adjust axis limits on the lateral KDE to match the main plot
y_hist.set_ylim(main_ax.get_ylim())

# Save the plot in high resolution
plt.savefig('E:\\Correct_Data_Official\\Official_Images\\volcano_plot_300dpi.png', dpi=300)
plt.savefig('E:\\Correct_Data_Official\\Official_Images\\volcano_plot_300dpi.tiff', dpi=300)

# Display the plot
plt.show()
